# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1: 
# Shahar Ashkenazi 
# 316244060
# Shaharke9@gmail.com


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [8]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [7]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [ ]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [9]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\E5470\Desktop\Assignment33


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [10]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [11]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [12]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [13]:
# Preprocessing the Data function

def preprocessing(text):
    text = re.sub(r'\d+', '', text) # removing all digits.
    text = re.sub(r'[^\w\s]', '', text) # removing all punctuation.
    text = re.sub(r'\s+', ' ', text) # replacing multi-space to one space.
    text = text.strip() # removing all spaces before and after text.

    text_preproccesed = ''

    tokens = ht.tokenize(text)
    for language, word, _, _ in tokens:
        if language == 'HEBREW':
            text_preproccesed += word + ' '
  
    return text_preproccesed

In [34]:
# Test
#print(preprocessing('##%^&*)__##@!!בדדיקה? מה, לראות! אם זה9 עובדY +או לא'))

In [17]:
# Applying preprocessing on both train & test DBs
df_train['story'] = df_train['story'].apply(preprocessing)
df_test['story'] = df_test['story'].apply(preprocessing)

# Changing labels to be 1 for male and 0 for female
df_train['gender'] = np.where(df_train['gender'] == 'm',1,0)

In [18]:
X_train= df_train['story']
y_train=df_train['gender']

In [20]:
models = [
    {
        'name': 'Perceptron',
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('norm',StandardScaler(with_mean = False)),
            ('clf', Perceptron())
        ]),
        'params': {
            'tfidf__ngram_range': [(1,1), (1,2)],
            'tfidf__min_df': [1,3,5],
            'clf__alpha': [0.001,0.01]}
        
        
        },
    {
        'name': 'SGDClassifier',
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('norm', StandardScaler(with_mean = False)),
            ('clf', SGDClassifier())
        ]),
        'params': {
            'tfidf__ngram_range': [(1,1), (1,2)],
            'tfidf__min_df': [1,3,5], 
            'clf__alpha': [0.001,0.01],
            'clf__penalty': ['l1', 'l2'],
         }     } , 
        
      {
          'name': 'LinearSVC',
          'pipeline': Pipeline([
              ('tfidf', TfidfVectorizer()),
              ('norm', StandardScaler(with_mean=False)),
              ('clf', LinearSVC())
          ]),
          'params': {
              'tfidf__ngram_range': [(1, 1), (1, 2)],
              'tfidf__min_df': [1,3,5],
              'clf__C': [0.1, 1.0],
              'clf__penalty': ['l1', 'l2']
          }
      }
    ]
        
      

best_model=None
best_F1score=0
for model in models:
    grid= GridSearchCV(model['pipeline'],param_grid=model['params'],cv=10,scoring='f1_macro')
    print('parameters: ', grid.get_params)
    grid.fit(X_train,y_train)
      
    if grid.best_score_ > best_F1score:
        best_F1score=grid.best_score_
        best_model=grid.best_estimator_
        best_model_name = model['name']
        
print(best_model)
print(best_model_name)

parameters:  <bound method BaseEstimator.get_params of GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm',
                                        StandardScaler(with_mean=False)),
                                       ('clf', Perceptron())]),
             param_grid={'clf__alpha': [0.001, 0.01],
                         'tfidf__min_df': [1, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')>


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm',
                                        StandardScaler(with_mean=False)),
                                       ('clf', Perceptron())]),
             param_grid={'clf__alpha': [0.001, 0.01],
                         'tfidf__min_df': [1, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')

parameters:  <bound method BaseEstimator.get_params of GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm',
                                        StandardScaler(with_mean=False)),
                                       ('clf', SGDClassifier())]),
             param_grid={'clf__alpha': [0.001, 0.01],
                         'clf__penalty': ['l1', 'l2'],
                         'tfidf__min_df': [1, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')>


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm',
                                        StandardScaler(with_mean=False)),
                                       ('clf', SGDClassifier())]),
             param_grid={'clf__alpha': [0.001, 0.01],
                         'clf__penalty': ['l1', 'l2'],
                         'tfidf__min_df': [1, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')

parameters:  <bound method BaseEstimator.get_params of GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm',
                                        StandardScaler(with_mean=False)),
                                       ('clf', LinearSVC())]),
             param_grid={'clf__C': [0.1, 1.0], 'clf__penalty': ['l1', 'l2'],
                         'tfidf__min_df': [1, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')>


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm',
                                        StandardScaler(with_mean=False)),
                                       ('clf', LinearSVC())]),
             param_grid={'clf__C': [0.1, 1.0], 'clf__penalty': ['l1', 'l2'],
                         'tfidf__min_df': [1, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_macro')

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=3, ngram_range=(1, 2))),
                ('norm', StandardScaler(with_mean=False)),
                ('clf', SGDClassifier(alpha=0.01))])
SGDClassifier


In [21]:
print(best_model)
print(best_model_name)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=3, ngram_range=(1, 2))),
                ('norm', StandardScaler(with_mean=False)),
                ('clf', SGDClassifier(alpha=0.01))])
SGDClassifier


In [23]:
import pickle

In [24]:
#Saving the best model
#with open('bestmodel.pkl', 'wb') as file: 
    #pickle.dump(best_model,file)

In [ ]:
with open('bestmodel.pkl', 'rb') as file:
    best_model = pickle.load(file)

In [26]:
print('the F1 score is: ' , best_F1score)

the F1 score is:  0.7067487167660944


In [27]:
# Improving the quality of the best model

sgd_model = {
    'name': 'SGDClassifier',
    'pipeline': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('norm', StandardScaler(with_mean=False)),
        ('clf', SGDClassifier())
    ]),
    'params': {
        'tfidf__ngram_range': [(1, 2)],
        'tfidf__min_df': [3],
        'clf__alpha': [0.001],
        'clf__penalty': ['l2'],
        'clf__max_iter': [1000, 2000],
        'clf__tol': [1e-3, 1e-4],
        'clf__loss': ['hinge', 'log'],
        'clf__eta0': [0.1, 0.01],
        'clf__learning_rate': ['optimal', 'adaptive'],
    }
}

best_sgd_model = None
best_sgd_F1score = 0

sgd_grid = GridSearchCV(sgd_model['pipeline'], param_grid=sgd_model['params'], cv=10, scoring='f1_macro')
sgd_grid.fit(X_train, y_train)

if sgd_grid.best_score_ > best_sgd_F1score:
    best_sgd_F1score = sgd_grid.best_score_
    best_sgd_model = sgd_grid.best_estimator_

print(best_sgd_model)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('norm',
                                        StandardScaler(with_mean=False)),
                                       ('clf', SGDClassifier())]),
             param_grid={'clf__alpha': [0.001], 'clf__eta0': [0.1, 0.01],
                         'clf__learning_rate': ['optimal', 'adaptive'],
                         'clf__loss': ['hinge', 'log'],
                         'clf__max_iter': [1000, 2000], 'clf__penalty': ['l2'],
                         'clf__tol': [0.001, 0.0001], 'tfidf__min_df': [3],
                         'tfidf__ngram_range': [(1, 2)]},
             scoring='f1_macro')

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=3, ngram_range=(1, 2))),
                ('norm', StandardScaler(with_mean=False)),
                ('clf',
                 SGDClassifier(alpha=0.001, eta0=0.01, loss='log',
                               max_iter=2000, tol=0.0001))])


In [28]:
# Saving the best SGD model
with open('bestSGDmodel.pkl', 'wb') as file:
    pickle.dump(best_sgd_model,file)

In [ ]:
with open('bestSGDmodel.pkl', 'rb') as file:
    best_sgd_model = pickle.load(file)

In [29]:
print('the updated F1 score is', best_sgd_F1score)

the updated F1 score is 0.7274024495045874


In [30]:
y_prediction = best_sgd_model.predict(df_test["story"])
y_prediction = np.where(y_prediction == 1, 'm','f')
pd.DataFrame({'id_test': df_test["test_example_id"], 'y_prediction': y_prediction}).head(5)
pd.DataFrame({'id_test': df_test["test_example_id"], 'y_predicion': y_prediction}).tail(5)

,id_test,y_prediction
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


,id_test,y_predicion
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


In [31]:
predictions = best_sgd_model.predict(df_test['story'])

# Print the predictions
print(predictions)

[1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1
 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1
 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1
 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [32]:
df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': y_prediction})
df_predicted.to_csv('classification_results.csv',index=False)